## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [2]:
! pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=776fad349cd7c79c1d0127721d3bf9247609c1406cee3dd709bdb7c0cefe4eda
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [4]:
! pip install lightgbm

In [5]:

! pip install catboost

     |████████████████████████████████| 67.3MB 62kB/s 


In [6]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [7]:
import os
os.listdir()

['.config', 'sample_data']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [8]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [9]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [10]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        |  7.663    |  2.308    |  5.355    |
|  2        |  9.971    |  2.984    |  6.987    |
|  3        |  8.632    |  2.32     |  6.312    |
|  4        |  8.0      |  1.0      |  7.0      |
|  5        |  5.0      |  1.0      |  4.0      |


Great, now let's print the best parameters and the associated maximized target.

In [11]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 2.984110105761294, 'b': 6.987308340718445}


9.971418446479738

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [12]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [13]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [14]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [15]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [16]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [19]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

In [41]:
full_df.shape

(200000, 33)

Apply the earlier defined feature engineering functions to the full dataframe.

In [21]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [22]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [23]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [25]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [26]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7434   |  0.01753  |  0.03454  |  44.09    |  2.801e+0 |  1.401e+0 |  98.02    |
|  2        |  0.7289   |  0.006125 |  0.04692  |  31.33    |  3.777e+0 |  1.035e+0 |  272.2    |
|  3        |  0.7293   |  0.02365  |  0.01867  |  36.57    |  112.0    |  1.045e+0 |  3.971e+0 |
|  4        |  0.7159   |  0.01167  |  0.04782  |  13.08    |  85.21    |  137.5    |  264.7    |
|  5        |  0.7159   |  0.02489  |  0.04376  |  43.17    |  841.2    |  290.9    |  2.089e+0 |
|  6        |  0.744    |  0.04613  |  0.01088  |  62.49    |  9.426e+0 |  1.991e+0 |  3.813e+0 |
|  7        |  0.7438   |  0.01371  |  0.04513  |  61.53    |  157.7    |  1.982e+0 |  158.4    |
|  8        |  0.7431   |  0.03548  |  0.02829  |  17.34    |  5.663e+0 |  1.724e+0 |  1.59e+03 |
|  9        |  0.743

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [27]:
lgbBO.max

{'params': {'lambda_l1': 0.046132049677931546,
  'lambda_l2': 0.010877237532350665,
  'max_depth': 62.485096496346095,
  'min_child_samples': 9425.965013626059,
  'min_data_in_leaf': 1990.585231071668,
  'num_leaves': 3812.6884762766263},
 'target': 0.7439603982677533}

Review the process at each step by using the '.res[0]' function.

In [28]:
lgbBO.res[0]

{'params': {'lambda_l1': 0.01753130551359572,
  'lambda_l2': 0.034544332680572096,
  'max_depth': 44.086163906507785,
  'min_child_samples': 2801.100991222532,
  'min_data_in_leaf': 1401.3092752469004,
  'num_leaves': 98.02429471102128},
 'target': 0.743366458819084}

In [29]:
lgbBO.res[1]

{'params': {'lambda_l1': 0.006125488820145398,
  'lambda_l2': 0.04691844475552487,
  'max_depth': 31.330067594526096,
  'min_child_samples': 3776.838832778533,
  'min_data_in_leaf': 1035.0515700369983,
  'num_leaves': 272.1604393524419},
 'target': 0.7289101006871085}

In [30]:
lgbBO.res[2]

{'params': {'lambda_l1': 0.023654524229893167,
  'lambda_l2': 0.01867076030035445,
  'max_depth': 36.567406431522436,
  'min_child_samples': 111.95219500133275,
  'min_data_in_leaf': 1045.4174509790128,
  'num_leaves': 3971.3209841500097},
 'target': 0.7293415751249995}

In [31]:
lgbBO.res[3]

{'params': {'lambda_l1': 0.011669966613845884,
  'lambda_l2': 0.04781754884605134,
  'max_depth': 13.080572819021722,
  'min_child_samples': 85.21416365152635,
  'min_data_in_leaf': 137.46485798051532,
  'num_leaves': 264.7475980496929},
 'target': 0.7158915268655903}

In [32]:
lgbBO.res[4]

{'params': {'lambda_l1': 0.024885390657031575,
  'lambda_l2': 0.0437568085412881,
  'max_depth': 43.172302748194085,
  'min_child_samples': 841.1553761436794,
  'min_data_in_leaf': 290.8598103726496,
  'num_leaves': 2088.6269530360237},
 'target': 0.7159331178565734}

In [33]:
lgbBO.res[5]

{'params': {'lambda_l1': 0.046132049677931546,
  'lambda_l2': 0.010877237532350665,
  'max_depth': 62.485096496346095,
  'min_child_samples': 9425.965013626059,
  'min_data_in_leaf': 1990.585231071668,
  'num_leaves': 3812.6884762766263},
 'target': 0.7439603982677533}

In [34]:
lgbBO.res[6]

{'params': {'lambda_l1': 0.013707923078799695,
  'lambda_l2': 0.045131931587928505,
  'max_depth': 61.53441915230901,
  'min_child_samples': 157.66852565039892,
  'min_data_in_leaf': 1982.4725163266264,
  'num_leaves': 158.40619012074757},
 'target': 0.7438109041572701}

In [35]:
lgbBO.res[7]

{'params': {'lambda_l1': 0.035482161194393715,
  'lambda_l2': 0.028289438432907328,
  'max_depth': 17.343502768951318,
  'min_child_samples': 5663.454633134905,
  'min_data_in_leaf': 1724.32008216995,
  'num_leaves': 1589.6680246702583},
 'target': 0.7430961005189071}

In [36]:
lgbBO.res[8]

{'params': {'lambda_l1': 0.02353089295372467,
  'lambda_l2': 0.015083148484051158,
  'max_depth': 46.638235844033716,
  'min_child_samples': 9857.106056307966,
  'min_data_in_leaf': 1804.3319542710358,
  'num_leaves': 219.17547316622068},
 'target': 0.7436085354436625}

In [37]:
lgbBO.res[9]

{'params': {'lambda_l1': 0.033845711619955664,
  'lambda_l2': 0.04127242531896128,
  'max_depth': 45.458938886563736,
  'min_child_samples': 4039.773976788412,
  'min_data_in_leaf': 1998.5847950358664,
  'num_leaves': 3845.842376328387},
 'target': 0.7435184703482068}

In [38]:
lgbBO.res[10]

{'params': {'lambda_l1': 0.013030950814881254,
  'lambda_l2': 0.012297773247983423,
  'max_depth': 54.72915988882846,
  'min_child_samples': 6814.01837584329,
  'min_data_in_leaf': 1985.545187745145,
  'num_leaves': 108.39263265329154},
 'target': 0.7436509958752612}

In [39]:
lgbBO.res[11]

{'params': {'lambda_l1': 0.02011214129602435,
  'lambda_l2': 0.032417968636818825,
  'max_depth': 11.404866013132237,
  'min_child_samples': 1830.2793444763317,
  'min_data_in_leaf': 1996.950569778038,
  'num_leaves': 1829.2437916863598},
 'target': 0.7437001222263492}